In [2]:
import numpy as np
#
# The function in this file follow closely after the MATLAB scripts
# written by Alessandro Foi and Markku Makitalo - Tampere University
# of Technology - 2011-2012
#
#    http://www.cs.tut.fi/~foi/invansc/
#
# that accompanies their paper:
#
#    Reference: M. Makitalo and A. Foi, "Optimal inversion of the
#    generalized Anscombe transformation for Poisson-Gaussian noise",
#    IEEE Trans. Image Process., doi:10.1109/TIP.2012.2202675


#-------- Variance stabilizing transforms -----------

def anscombe(x):
    '''
    Compute the anscombe variance stabilizing transform.
      the input   x   is noisy Poisson-distributed data
      the output  fx  has variance approximately equal to 1.
    Reference: Anscombe, F. J. (1948), "The transformation of Poisson,
    binomial and negative-binomial data", Biometrika 35 (3-4): 246-254
    '''
    return 2.0*np.sqrt(x + 3.0/8.0)

def inverse_anscombe(z):
    '''
    Compute the inverse transform using an approximation of the exact
    unbiased inverse.
    Reference: Makitalo, M., & Foi, A. (2011). A closed-form
    approximation of the exact unbiased inverse of the Anscombe
    variance-stabilizing transformation. Image Processing.
    '''
    #return (z/2.0)**2 - 3.0/8.0
    return (1.0/4.0 * np.power(z, 2) +
            1.0/4.0 * np.sqrt(3.0/2.0) * np.power(z, -1.0) -
            11.0/8.0 * np.power(z, -2.0) + 
            5.0/8.0 * np.sqrt(3.0/2.0) * np.power(z, -3.0) - 1.0 / 8.0)

def generalized_anscombe(x, mu, sigma, gain=1.0):
    '''
    Compute the generalized anscombe variance stabilizing transform,
    which assumes that the data provided to it is a mixture of poisson
    and gaussian noise.
    The input signal  z  is assumed to follow the Poisson-Gaussian noise model
        x = gain * p + n
    where gain is the camera gain and mu and sigma are the read noise
    mean and standard deviation.
    We assume that x contains only positive values.  Values that are
    less than or equal to 0 are ignored by the transform.
    Note, this transform will show some bias for counts less than
    about 20.
    '''
    y = gain*x + (gain**2)*3.0/8.0 + sigma**2 - gain*mu

    # Clamp to zero before taking the square root.
    return (2.0/gain)*np.sqrt(np.maximum(y, 0.0))

def inverse_generalized_anscombe(x, mu, sigma, gain=1.0):
    '''
    Applies the closed-form approximation of the exact unbiased
    inverse of Generalized Anscombe variance-stabilizing
    transformation.
    The input signal x is transform back into a Poisson random variable
    based on the assumption that the original signal from which it was
    derived follows the Poisson-Gaussian noise model:
        x = gain * p + n
    where gain is the camera gain and mu and sigma are the read noise
    mean and standard deviation.
    Roference: M. Makitalo and A. Foi, "Optimal inversion of the
    generalized Anscombe transformation for Poisson-Gaussian noise",
    IEEE Trans. Image Process., doi:10.1109/TIP.2012.2202675
    '''
    test = np.maximum(x, 1.0)
    exact_inverse = ( np.power(test/2.0, 2.0) +
                      1.0/4.0 * np.sqrt(3.0/2.0)*np.power(test, -1.0) -
                      11.0/8.0 * np.power(test, -2.0) +
                      5.0/8.0 * np.sqrt(3.0/2.0) * np.power(test, -3.0) -
                      1.0/8.0 - np.power(sigma, 2) )
    exact_inverse = np.maximum(0.0, exact_inverse)
    exact_inverse *= gain
    exact_inverse += mu
    exact_inverse[np.where(exact_inverse != exact_inverse)] = 0.0
    return exact_inverse

#def inverse_generalized_anscombe(y,mu,sigma,gain=1.0):
#    return (1.0/gain)*(gain*y/2.0)**2 - gain*3.0/8.0 - (sigma**2)/gain + mu

In [3]:
from bm3d import bm3d, BM3DProfile
import numpy as np
import scipy.misc
from matplotlib import pyplot as plt
from tifffile import imread, imsave

In [4]:
path='/Users/prakash/Downloads/Mouse actin/'

# Load the test data
noisy=imread(path+"sample_attempt2.tif")[:,:,:512].astype(np.float32)
# We are loading only a sub image to spped up computation

# We estimate the ground truth by averaging.
clean=np.mean(noisy[:,...],axis=0)[np.newaxis,...]

print(noisy.shape)
print(clean.shape)


(100, 1024, 512)
(1, 1024, 512)


In [5]:
def PSNR(gt, img, rangePSNR):
    '''
    Compute PSNR.
    Parameters
    ----------
    gt: array
        Ground truth image.
    img: array
        Predicted image.''
    '''
    mse = np.mean(np.square(gt - img))
    return 20 * np.log10(rangePSNR) - 10 * np.log10(mse)

In [ ]:
offset=0
from tqdm import tqdm
psnrs=[]

for img_idx in range(noisy.shape[0]):
    print("--------------------imgae:", img_idx, " ------------------------------------")
    denoisedImgs=[]
    mses=[]
    bestMSE = None
    bestImg = None
    bestSig= None
    
    for s in tqdm(range(1)):
#         sig = (s+1)*2.5
        sig = 2*2.5

        img=noisy[img_idx,...]
        cl= clean[0]

        
        ans=anscombe(img-offset)
        denoised = inverse_anscombe (bm3d (ans,sig)) + offset


        mse = np.mean((cl-denoised)**2)
        mses.append(mse)
        denoisedImgs.append(denoised.copy())

        if bestMSE is None or (mse<bestMSE):
            bestImg = denoised.copy()
            bestMSE = mse
            bestSig = sig
#         print (sig, bestSig, mse) 
    rangePSNR = np.max(cl)-np.min(cl)
    psnr = PSNR(cl, bestImg, rangePSNR)
    psnrs.append(psnr)
    print("image:", img_idx, "\t PSNR:", psnr, "\t mean PSNR:", np.mean(psnrs))


--------------------imgae: 0  ------------------------------------


100%|██████████| 1/1 [00:11<00:00, 11.91s/it]


image: 0 	 PSNR: 33.78143171184871 	 mean PSNR: 33.78143171184871
--------------------imgae: 1  ------------------------------------


100%|██████████| 1/1 [00:11<00:00, 11.60s/it]


image: 1 	 PSNR: 33.74521790083202 	 mean PSNR: 33.763324806340364
--------------------imgae: 2  ------------------------------------


...

100%|██████████| 1/1 [00:14<00:00, 14.91s/it]


image: 69 	 PSNR: 33.72240913205337 	 mean PSNR: 33.64290891489719
--------------------imgae: 70  ------------------------------------


100%|██████████| 1/1 [00:15<00:00, 15.92s/it]


image: 70 	 PSNR: 33.43434385561672 	 mean PSNR: 33.63997137885099
--------------------imgae: 71  ------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]